# 0. Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import utils as util

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# 1. Preamble analysis of the dataset

In [ ]:
df = pd.read_csv("./data/covid_data.csv")
df.head()

### 1.1 Analysing existing data

In [ ]:
df_null = df.copy()
for i in [97, 98, 99]:
   df_null.replace(i , np.nan, inplace = True)

In [ ]:
df_null.isnull().sum()

As we can show with the following graph, there are **a lot** of NA values in the dataset. We will have to handle them.

One approach is to take the mean of the column and replace the NA values with it. However, this is not a good approach, as it will skew the data. We will have to find a better way to handle the NA values.

We will therefore have to go each problematic columun to fix the eventual problematic data

Remember that in the dataset definition it says that 97,98 and 99 are null values, let's deal with that

In [ ]:
# sns.heatmap(df_null.isnull(), cbar=False)
# plt.title('Before data cleanup', color = 'black', fontsize = 15)
# plt.show()

#### As we can see, that's no bueno...

In [ ]:
df.describe().round(3).T.drop('count', axis = 1)

Furthermore, we have some suspiciously skewed data. Is really half of the population pregnant?!

### 1.2 Managing the DEAD people 

People with a DATE_DIED value of 9999-99-99 simply aren't dead, so we'll just create a new DEAD column to represent that

In [ ]:
# Check for any strings in the feature "DATE_DIED"
df['DATE_DIED'][df['DATE_DIED'].apply(lambda x: isinstance(x, str))]

In [ ]:
df['DEAD'] = [2 if i=='9999-99-99' else 1 for i in df.DATE_DIED]

In [ ]:
df['DEAD'].value_counts(normalize=True)

We'll also replace 9999-99-99 with NaN for the time being

In [ ]:
df['DATE_DIED'].replace('9999-99-99', np.nan, inplace = True)

In [ ]:
df['DATE_DIED']

And we can convert the date in an actual datetime object

In [ ]:
df['DATE_DIED'] =  pd.to_datetime(df['DATE_DIED'], format='%d/%m/%Y', errors='coerce')

In [ ]:
df['DATE_DIED'].isnull().sum()

#### How does the data look now ?

In [ ]:
df.describe().round(3).T.drop('count', axis = 1)


Hmmm, it looks like we'll have to work on the PREGNANT, ICU, and INTUBED people

### 1.3 SEX Values

We'll just set the 1 and 2 values to "Female" and "Male" respectively

In [ ]:
df["SEX"] = ["M" if i == 2 else "F" for i in df["SEX"]]
df.head()

### 1.4 Pregnant values

In [ ]:
df.SEX.value_counts()

In [ ]:
df.SEX.shape

Pregnant females ?

In [ ]:
df[(df['SEX'] == "F")]['PREGNANT']

In [ ]:
df[(df['SEX'] == "F")]['PREGNANT'].value_counts()

Pregnant males ?

In [ ]:
df[(df['SEX'] == "M")]['PREGNANT']

In [ ]:
df[(df['SEX'] == "M") & (df['PREGNANT'])]['PREGNANT'].value_counts()

In [ ]:
df['PREGNANT'].value_counts()

It looks like 97 indicates males that aren't pregnant. In other words, for those values we can just input 2 instead of 97

In [ ]:
df['PREGNANT'].replace (97, 2, inplace = True)

Finally, 98 represents the females that are unknown to be pregnant or not

In [ ]:
df['PREGNANT'].value_counts()

We can simply mark then as NA as the data is logically Not Available

In [ ]:
df['PREGNANT'].replace(98, None, inplace = True)
df['PREGNANT'].value_counts(normalize=True)

Most people aren't pregnant, this now makes a lot more sense

### 1.5 ICU values

In [ ]:
df.ICU.value_counts()

In [ ]:
pd.crosstab(df['PATIENT_TYPE'], df['ICU'])

From the above we can see that the missing values of 97 are all corresponding to the values of PATIENT_TYPE = 1 which is for non hospitalized patients, while those of 99 are the missing values of the hospitalized patients, which again can not be told or predicted.

So we can replace all the values of (97) with (2); since obviously patients who have never been hospitalized couldn't possibly be admitted to the ICU.

In [ ]:
df['ICU'].replace (97, 2, inplace = True)

In [ ]:
df.ICU.value_counts()

### 1.6 INTUBED values

In [ ]:
df.INTUBED.value_counts()

In [ ]:
pd.crosstab(df['PATIENT_TYPE'], df['INTUBED'])

Same logic of the ICU patients: patients that are intubed necessarily are also hospitalized

In [ ]:
df['INTUBED'].replace (97, 2, inplace = True)

In [ ]:
df.INTUBED.value_counts()

We'll replace everything that we can't infer with NaN

In [ ]:
for i in [98, 99]:
   df.replace(i , np.nan, inplace = True)

#### How does the data look now ?

In [ ]:
# First, let's temporarily reverse the step we did on "DATE_DIED" feature; as they are not really missing:
df_null2 = df.copy()
df_null2.DATE_DIED = df_null2.DATE_DIED.fillna("9999-99-99")

#Let's check again for our missing values:
df_null2.isnull().sum()

In [ ]:
df.describe().round(3).T.drop('count', axis = 1)

Looks much better already !
What about a heatmap ?

In [ ]:
# sns.heatmap(df_null2.isnull(), cbar=False)
# plt.title('After Data cleanup', color = 'black', fontsize = 15)
# plt.show()

### 1.7 Hospitalization

Hospitalization is described by the `PATIENT_TYPE` column. It has a value of either 1: at home or 2: in hospital. We can change this column to a boolean column which, instead of describing the patient type, will describe if the patient is hospitalized or not. That means that we will have to change the column name to `HOSPITALIZED`, but we'll also have to invert all the values.

In [ ]:
df["HOSPITALIZED"] = [1 if i == 2 else 2 for i in df["PATIENT_TYPE"]]
df[["HOSPITALIZED", "PATIENT_TYPE"]].head()

We'll drop the column as it is now redundant

In [ ]:
df = df.drop("PATIENT_TYPE", axis=1)
df.head()

### 1.8 Readability fix

As we can see, the column names are not very readable, so we'll just fix that. In the original datasheet, it is said that the boolean values, 1 and 2, are actually "Yes" and "No" respectively. We'll just change that as well to make them boolean.

In [ ]:
# Get columns whose data unique count is equal to 2 and are either 1, 2 or NA
binary_cols = [col for col in df.columns if df[col].nunique() == 2 and df[col].dropna().value_counts().index.isin([1,2]).all()]
binary_cols

The number of boolean columns described in the datasheet is 15, but we have 16. The culprit is the `USMER` column, which does have only two values, but do not describe a boolean value. We'll just remove it from the list.

In [ ]:
binary_cols.remove("USMER")
binary_cols

Now that we have a list of boolean value columns, we can change the values to strings of either "Y" or "N" and then change the column type to categorical.

In [ ]:
# Change the values of the binary columns to "Y" if 1, "N" if 2
for col in binary_cols:
    df[col] = df[col].replace({1: "Y", 2: "N"})
df.describe().round(3).T.drop('count', axis = 1)

### 1.9 Categorizing the columns

In [ ]:
# For every columns, display the number of unique values
df.nunique()

In [ ]:
non_categorical = ["DATE_DIED", "AGE"]
categorical = df.columns.drop(non_categorical)

for category in categorical:
    df[category].astype("category")
    
df.describe().round(3).T.drop('count', axis = 1)

# 2. Data appetizers with PCA analysis

In this section we will try to get a better understanding of the data by using PCA analysis. This approach will allow us to see the data in a 2D space, but mostly to see the correlation between the different features.

This will help us decide next which features we want to keep and which we want to drop, but also which machine learning algorithm we want to use.

In [ ]:
X_data = df.drop('DATE_DIED', axis=1)
len(X_data.columns)

In [ ]:
X_data = pd.get_dummies(X_data, columns=categorical)
len(X_data.columns)

In [ ]:
pca_scaler = MinMaxScaler()
X_data = pd.DataFrame(pca_scaler.fit_transform(X_data), columns=X_data.columns, index=X_data.index)
X_data

As the PCA doesn't take NaN values, we will have to find a way to deal with them. As we're only doing PCA analysis, we can simply take the average of the column and replace the NaN values with it.

In [ ]:
X_data.fillna(X_data.mean(), inplace=True)
X_data

In [ ]:
pca = PCA(n_components=X_data.shape[1])
x_new = pca.fit_transform(X_data)

In [ ]:
util.plot_pca_features(pca, X_data)
plt.show()